##Downloading libraries 

In [ ]:
#Installing transformers library for different hugging face models and tokenizers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.0 MB/s eta 0:00:00


In [ ]:
#T5 uses sentence piece, which needs to be explicitly installed on Colab 
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.3 MB/s eta 0:00:00


##Importing the Translation Models

We are performing back translation task which has 2 sub-steps: \\
1) Translation form English to French \\
2) Back translate to English 

Both of the steps require different Machine Translation models. We utilize a Marian Machine translation model, that was fined tuned for the specific language translation by Helsinki Lab.

In [ ]:
from transformers import pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")
back_translator = pipeline("translation", model ="Helsinki-NLP/opus-mt-fr-en")
input = "On the model's page here there's a Use in Transformers link that you can use to see the code to load it in their transformers package"

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


##Performing Back Translation and Converting the generation in a proper format

In [ ]:
#Converting the data into a proper format for fine tuning of the T5 model for COnditional Generation
def convert_to_t5_format(translator, back_translator, x):
  input = x
  if len(x.split())>250:
    input = ' '.join(x.split()[:250])
  output = back_translation(translator, back_translator,input)
  return ' '.join(('Input: '+ output.lower() + " Output: " + input).split()[:500])

In [ ]:
#Performing the Back Translation 
def back_translation(translator, back_translator, input):
  return back_translator(translator(input)[0]['translation_text'])[0]['translation_text']

In [ ]:
print(back_translation(translator, back_translator, input))

On the template page here there is a Use link in Transformers that you can use to see the code to load it in their transformer package


In [ ]:
# Load the TST files that were created earlier 
import pandas as pd
filename = "Taylor"
dataset = pd.read_csv(f'/content/drive/MyDrive/685 Project/{filename}_TST/{filename}TST.csv')

In [ ]:
dataset.head()

,content
0,he said the way my blue eyes shined
1,put those georgia stars to shame that night
2,"i said, ""that's a lie"""
3,just a boy in a chevy truck
4,that had a tendency of gettin' stuck


In [ ]:
print(convert_to_t5_format(translator, back_translator, dataset['content'].iloc[0]))

Input: be sure to listen and watch donald trick late in the night with david letterman as he presents the list of the top ten tonight! Output: be sure to tune in and watch donald trump on late night with david letterman as he presents the top ten list tonight!


In [ ]:
# Implement the Back translation on the dataset 
dataset = dataset['content'].apply(lambda x: convert_to_t5_format(translator, back_translator, x))

Your input_length: 511 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)
Your input_length: 511 is bigger than 0.9 * max_length: 512. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


In [ ]:
#Save the dataset, for future T5 fine tuning 
dataset.to_csv(f'/content/drive/MyDrive/685 Project/{filename}_TST/{filename}_BT_TST.csv')